<a href="https://colab.research.google.com/github/Sharaborina/algorithms/blob/master/Main_ChatBot_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y spacy --quiet
!pip install python-telegram-bot nltk geopy dateparser datefinder spacy deep_translator detectlanguage geotext 
!python -m spacy download en_core_web_lg >out 2>log
!python -m spacy download da_core_news_lg >out 2>log
from google.colab import drive
drive.mount('/content/drive')


     |████████████████████████████████| 460kB 7.8MB/s 
     |████████████████████████████████| 286kB 47.3MB/s 
     |████████████████████████████████| 12.8MB 54.2MB/s 
     |████████████████████████████████| 2.0MB 44.6MB/s 
     |████████████████████████████████| 61kB 5.8MB/s 
     |████████████████████████████████| 1.1MB 42.1MB/s 
     |████████████████████████████████| 9.1MB 40.2MB/s 
     |████████████████████████████████| 460kB 46.8MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 122kB 54.8MB/s 
  Created wheel for detectlanguage: filename=detectlanguage-1.5.0-cp37-none-any.whl size=3173 sha256=2cf61d7e94a242806261e22fd102e7d56da17f817a0b6ad73dd551d28cdf65a9
  Stored in directory: /root/.cache/pip/wheels/9e/30/66/53f3628609941078e5a3fb763a78010dbd64b996cac518b15b
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=b58bf7e5cffd8b6b555366ed35c12f9b9484da29287a53e882cd66a943460024
  Stored in

In [57]:
from telegram.ext import *
import telegram
import random
import pandas as pd
import requests
import geopy
import torch
import re
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from geopy.geocoders import Nominatim
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
#for intention classification
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
#telegram bot logging/debugging
import logging
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

#for date analyser
# from date_extractor import extract_dates
import dateparser #text to date
from datetime import timedelta
from datetime import datetime
import datefinder

#for geo extraction
from geotext import GeoText #only in English

# NLU
import nltk
import spacy
# translate replicas
from deep_translator import GoogleTranslator, single_detection
# detect language
# import detectlanguage
import itertools




In [44]:
#bot token
BOT_KEY = '1880919831:AAEJFa4yGm57IKr68qXEHYS7t1nS_LnX8iw'

NER_MODEL_DIR='/content/drive/MyDrive/ColabNotebooks/Spacy_NER_Model'

GENERATIVE_MODEL_DIR='/content/drive/MyDrive/ColabNotebooks/Generative_model'
# data directory
DATA_DIR = '/content/drive/MyDrive/ColabNotebooks/MultiWoz/'
# vocabulary filename
VOCAB_FILE = 'en_50k_pruned.subword'
# vocabulary file directory
VOCAB_DIR = '/content/drive/MyDrive/ColabNotebooks/MultiWoz/'
VOCAB_SIZE = 50000

# load a cleaned translated train data from file DATA_DIR/TRAIN_FILENAME, 
# otherwise  generate a train data from DIALOGUE_ENG_DB
LOAD_TRAIN_DATA = False
# train filename
TRAIN_FILENAME = 'label_and_train_data.json'
# path to pretrained NER_model
NER_MODEL_DIR = '/content/drive/MyDrive/ColabNotebooks/Spacy_NER_Model'

GENERATIVE_MODEL_SH = '/content/drive/MyDrive/ColabNotebooks/Generative_model_Sharaborina_state_dict.pt'


USE_PRETRAINED_MODEL = True #use pretrained model, otherwise start from a blank model
LOAD_MODEL = True #load pretrained model
TRAIN = True
N_LAYERS = 6
TRAIN_STEPS = 100



# Booking API
First let's create a `geoloc` function to find out latitude and longitude of a city/town/village, etc. Because Booking API works only with such coordinates.

In [4]:
def geoloc(city):
    geolocator = Nominatim(user_agent= 'sharaborinaly')
    location = geolocator.geocode(city)
    result = str(location.latitude-0.3)+'%2C'+str(location.latitude+0.3)+'%2C'+str(location.longitude-0.3)+'%2C'+str(location.longitude+0.3)
    return result

`booking_request` function gives us scraped data from the booking site. It takes required fileds: the name of city, arrival and departure dates, the number of guests.
Also there is not mandatory fields: the number of children, number of rooms, number of guests

In [5]:
def booking_request(city, dates = None, rooms = 1, guest = 1, children = 0):

    dates = sorted(dates)
    arrival_date = str(dates[0].date())
    departure_date = str(dates[1].date())

    if arrival_date == None or departure_date == None:
      arrival_date = datetime.today()
      departure_date = arrival_date + 1

    list_by_map_url = "https://apidojo-booking-v1.p.rapidapi.com/properties/list-by-map"

    list_by_map_querystring = {
        "search_id":"none",
        "children_age":"",
        "price_filter_currencycode":"EUR",
        "languagecode":"da",
        "travel_purpose":"leisure",
        "categories_filter":"class%3A%3A1%2Cclass%3A%3A2%2Cclass%3A%3A3",
        "children_qty":children,
        "order_by":"popularity",
        "guest_qty": guest,
        "room_qty": rooms,
        "departure_date": departure_date,
        "bbox":"",
        "arrival_date": arrival_date
    }
    
    list_by_map_querystring['bbox'] = geoloc(city) 
    list_by_map_headers = {
        'x-rapidapi-host': "apidojo-booking-v1.p.rapidapi.com",
        'x-rapidapi-key': "54d78e5c98mshd2b3a25b4adf763p1674f2jsnb7535d954daf"
    }

    list_by_map_response = requests.request("GET", list_by_map_url, headers=list_by_map_headers, params=list_by_map_querystring).json()
    result = []
    # prdic(list_by_map_response['result'])
    try:
      for list_result in list_by_map_response['result']:
          result.append({'hotel_id': list_result['hotel_id'],
                        'hotel_name': list_result['hotel_name'],
                        'url':list_result['url'],
                        'main_photo_url' : list_result['main_photo_url'],
                        'min_total_price': list_result['min_total_price'],
                        'address': list_result['address'],
                        'review_score': list_result['review_score'],
                        'currencycode' : list_result['currencycode']
                        })
    except:
      pass    
   
    if len(result) == 0:
      return 'Undskyld! Jeg kan ikke finde hoteller til de givne datoer. Prøv venligst andre datoer.' 
    elif len(result) <= 5:
      return result
    else:
      return result[:5]
        

In [7]:
# !pip install print_dict
# from print_dict import pd as prdic
# # Let's check the output
booking_request('Moskva', [datetime.today(), datetime.today() + timedelta(days=3)])

2021-05-30 18:05:00,374 - geopy - DEBUG - Nominatim.geocode: https://nominatim.openstreetmap.org/search?q=Moskva&format=json&limit=1
2021-05-30 18:05:00,478 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): apidojo-booking-v1.p.rapidapi.com:443
2021-05-30 18:05:01,937 - urllib3.connectionpool - DEBUG - https://apidojo-booking-v1.p.rapidapi.com:443 "GET /properties/list-by-map?search_id=none&children_age=&price_filter_currencycode=EUR&languagecode=da&travel_purpose=leisure&categories_filter=class%253A%253A1%252Cclass%253A%253A2%252Cclass%253A%253A3&children_qty=0&order_by=popularity&guest_qty=1&room_qty=1&departure_date=2021-06-02&bbox=55.4504461%252C56.050446099999995%252C37.3174943%252C37.917494299999994&arrival_date=2021-05-30 HTTP/1.1" 200 52504


[{'address': 'Izmailovskoe Shosse 71, 4 G - D',
  'currencycode': 'RUB',
  'hotel_id': 245721,
  'hotel_name': 'Izmailovo Gamma Hotel',
  'main_photo_url': 'https://cf.bstatic.com/xdata/images/hotel/square60/287211548.jpg?k=8d1614b23879d4250db3c1a4e5b86d5d888888c89449ac311f700b203bdc2bc1&o=',
  'min_total_price': 6600,
  'review_score': 8.7,
  'url': 'https://www.booking.com/hotel/ru/izmailovo-gamma.html'},
 {'address': 'Izmailovskoye Shosse 71 Bld.2B',
  'currencycode': 'RUB',
  'hotel_id': 269458,
  'hotel_name': 'Izmailovo Beta Hotel',
  'main_photo_url': 'https://cf.bstatic.com/xdata/images/hotel/square60/90460449.jpg?k=d523769f5f5e3a11321fb347eec460eafedbd875c098297fd4f4477e8b1deafd&o=',
  'min_total_price': 10483,
  'review_score': 8.1,
  'url': 'https://www.booking.com/hotel/ru/izmaylovo-beta.html'},
 {'address': 'Ulitsa Bakhrushina 11',
  'currencycode': 'RUB',
  'hotel_id': 803100,
  'hotel_name': 'Ibis Moscow Centre Bakhrushina',
  'main_photo_url': 'https://cf.bstatic.com/xd

# Intention classifier
Here let's build a simple classifier of an input phrase.
But first let us create BOT_CONFIG - it is a dictionary with intents, which have their phrase examples and some prepared answers. 
We will classify an input phrase by intent, and if there will be returned some intent, the algorithm will use a prepared response. If not - a failure phrase will be used.

In [8]:
BOT_CONFIG = {
    'intents':{
        'hello':{'examples': ['hej', 'dav', 'hi', 'hello', 'goddag','hej bot', 'dav bot', 'hi bot', 'hello bot', 'goddag bot', 'hej du'], 'responses': ['Hej! Hvordan kan jeg hjælpe dig?', 'Goddag! Hvordan kan jeg hjælpe dig?', 'Hey! Welcome! Hvordan kan jeg hjælpe dig?']},
        'bye':{'examples': ['hej-hej', 'bye', 'ha en god dag', 'bye bot', 'tak hav en god dag', ' hav en god dag', 'ha det godt', 'bye-bye'], 'responses': ['Jeg håber du vil have en vidunderlig dag!','Jeg hjælper gerne. Hav en god dag!','Hej-hej!', 'Hav en rigtig god dag!', 'Farvel og tak!', 'Farvel!']},
        'name':{'examples': ['hvordan hedder du', 'hvad hedder du', 'hvad er dit navn', 'dit navn', 'fortæl mig dit navn', 'fortæl dit navn'], 
                'responses': ['Jeg hedder AirbnbBot, jeg kan hjælpe dig med at reservere et hotelværelse. Kan du venligst give mig følgende information: en by, en ankomsttid, en afgangstid. Og jeg vil finde et hotelværelse', 'Jeg er AirbnbBot, jeg kan reservere et hotelværelse til dig. Kan du venligst give mig følgende information: en by, en ankomsttid, en afgangstid. Og jeg vil finde et hotelværelse']},
        'sorry': {'examples': ['undskyld', 'det må du undskyld', 'med fejl', 'min fejl', 'det er med fejl', 'det er min fejl', 'undskyld bot', 'oh undskyld', 'undskyld for det'], 'responses': ['Det er helt i orden', 'Det er i orden', 'Du behøver ikke at undskylde dig']},
        'help': {'examples': ['jeg vil book et billigt hotel med parkering', 'vi leder efter et hotel med parkering', 'vil have et hotel med parkering', 'vil book et hotel med parkering','jeg vil book et hotel med parkering', 'jeg leder efter et billigt hotel', 'jeg leder efter et hotel med parkering', 'jeg har brug for et hotel', 'jeg leder efter et hotel i københavn','jeg leder efter et hotel i buenos aires',
                              'jeg leder efter et hotel i new york','jeg leder efter et hotel i moskva','jeg vil book et hotel i amsterdam', 'jeg vil book et hotel i buenos aires', 'jeg vil book et hotel i bangkok','jeg vil book et hotel i luxenburg', 'jeg vil book et hotel i københavn','jeg vil book et hotel','kan du book et hotel til mig', 'kan du booke hotel', 'jeg har også brug for at finde et sted at bo',
                              'jeg leder efter et gæstehus i centrum af byen', 'jeg har brug for et sted at bo syd for byen', 'jeg leder efter oplysninger om et hotel i','kan du venligst hjælpe mig med at finde et sted at bo','jeg leder efter et hotel at bo på i centrum, kan du slå dette op for mig','jeg leder efter et bestemt hotel','jeg leder efter et sted at bo, der har en stjerne på ','jeg leder efter informationom hoteler i',
                              'jeg leder efter et hotel i ','venligst book et hotel','jeg har brug for et sted at bo','kan du hjælpe dig med noget','jeg kunne ikke booke et hotel','jeg vil booke et hotel','jeg er på udkig efter et sted at bo','jeg vil reservere et hotel', 'reserver et hotelværelse i', 'kan du reservere et hotel', 'kan du reservere et hotelværelse', 'kan du reservere et hotelværelse for mig', 'kan du hjælpe mig', 'kan du hjælpe mig med at reservere',
                              'jeg har brug for hjælp', 'jeg har brug for din hjælp', 'hjælp mig med det'], 
                 'responses': ['Jeg vil gerne hjælpe dig. Jeg kan reservere et hotelværelse til dig. Kan du venligst give mig følgende information: en by, en ankomsttid, en afgangstid. Og jeg vil finde et hotelværelse', 'Jeg vil gerne reservere et hotelværelse til dig. Kan du venligst give mig følgende information: en by, en ankomsttid, en afgangstid. Og jeg vil finde et hotelværelse']},
        'answer': {'examples': ['jeg planlægger en tur til','jeg skal rejse til', 'jeg skal til', 'jeg skal til d og tilbage d', 'jeg skal til d', 'vi skal til', 'vi vil rejse til', 'vi skal rejse til'], 
                   'responses': ['Det er fint, tak! Kan du venligst give mig følgende information: en by, en ankomsttid, en afgangstid. Og jeg vil finde et hotelværelse til dig.']},
        'bad words':{'examples': ['bland dig udenom', 'hold kæft', 'hold kæft bot','din kælling', 'dit røvhul', 'for fanden', 'for fanden bot', 'for helvede'], 'responses': ['Du skal ikke sværge her. Jeg er en anstændig bot.', 'Vær så venlig ikke at sværge. Jeg er en anstændig bot.']},
        'how are you':{'examples': ['er det i orden', 'hvordan går det', 'hvordan går det med dig', 'hvordan har du det', 'hvordan fyler du dig'], 
                       'responses': ['Det går meget dodt, tak! Jeg er klar til at hjælpe dig med at reservere et hotelværelse. Kan du venligst give mig følgende information: en by, en ankomsttid, en afgangstid. Og jeg vil finde et hotelværelse', 'Jeg har det godt, tak! Jeg er klar til at hjælpe dig med at reservere et hotel værelse. Kan du venligst give mig følgende information: en by, en ankomsttid, en afgangstid. Og jeg vil finde et hotelværelse', 
                                     'Godt, mange tak! Jeg vil gerne hjælpe dig med at reservere et hotel værelse. Kan du venligst give mig følgende information: en by, en ankomsttid, en afgangstid. Og jeg vil finde et hotelværelse']},
        'thanks':{'examples': ['tak for hjælpen', 'okey mange tak','okej tak','okay tak','det er det tak','tak det er alt', 'tak for din hjælp', 'tak', 'mange tak', 'det ville være alt tak'], 'responses': ['Jeg er glad for, at jeg var i stand til at hjælpe','Du er velkommen. Tak!', 'Tak! Glad for at kunne hjælpe', 'Du er meget velkommen!']},
        
    },
    'failure_phrase': ['Undskyld, jeg kan ikke forstår det. Kan du skrive det igen? Kan du venligst give mig følgende information: en by, en ankomsttid, en afgangstid. Og jeg vil finde et hotel til dig.', 'Undskyld, jeg studerer bare at arbejde som bot og jeg kan ikke forstår nogle ting. Kan du venligst give mig følgende information: en by, en ankomsttid, en afgangstid. Og jeg vil finde et hotel til dig.']
    
}

Then we construct a long vector of input data (`X_text`) and corresponding output (`y`).

In [9]:
X_text = []
y = []
for intent, intent_data in BOT_CONFIG['intents'].items():
  for example in intent_data['examples']:
    X_text.append(example)
    y.append(intent)


Vectorization from a text to a number vector. For each word, the algorithm constructs a column in a matrix (by default `analyzer='word'`).
But the experiments shows that n-gram method works better. Let's take 3-gram method. For example, `'Hello, world'` gives a list of subwords (3-grams) 

`['Hel', 'ell', 'llo', 'wor', 'orl', 'rld']` 

In [10]:
# vectorizer = CountVectorizer()
# vectorizer = TfidfVectorizer()
vectorizer = CountVectorizer(analyzer='char', ngram_range=(3,3))
# vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(3,3))

X = vectorizer.fit_transform(X_text)

#let's shuffle before splitting into validational and
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

Let's use Linear classifier `LogisticRegression`

In [11]:
#training of the model
# clf = LogisticRegression(random_state = 0)
clf = LinearSVC(random_state = 0)
clf.fit(X,y)

#predict function
def classification(rep):
  prediction = clf.predict(vectorizer.transform([rep]))
  # print(prediction)
  return prediction[0]

In [12]:
# Estimation of quality
from sklearn.metrics import precision_score
y_pred = clf.predict(X_test)
precision_score(y_test, y_pred, average='micro')
# clf.score(X_test, y_test)

1.0

In [13]:
#Let's check
classification('hej kan du hjæelpe mig'), classification('bvvvvvvvv')

('help', 'hello')

Let us preprocess a input phrase - it is needed to transform it to lower case and remove sybmols

In [14]:
def clear(rep):
    rep = rep.lower()
    alphabet = '0123456789abcdefghijklmnopqrstuvwxyzåøæ- '
    result = ''.join(symbol for symbol in rep if symbol in alphabet)
    return result

In [15]:
def get_failure_phrase():
    return random.choice(BOT_CONFIG['failure_phrase'])

def too_many_enity_failure_phrase():
    answer = 'Du har skrevet mere oplystninger end det var nødvendigt. Vær så venlig at give mig disse oplystninger: en by, en ankomsttid, en afgangstid.'
    return answer

And it is also necessary to add calculation of the Levenshtein edit-distance between two strings. It will help to take attention to small mistakes.


In [16]:
def classify_intent(rep):
    rep = clear(rep)
    intent = classification(rep) # classification algorithm always gives us an intention

    # here we check the distance between answers of the intent and a user's replica
    for intent, intent_data in BOT_CONFIG['intents'].items():
        for example in intent_data['examples']:
            example = clear(example)
            distance = nltk.edit_distance(rep, example)
            if example and distance/ len(example) < 0.45:
              return intent



And if intent is returned by the function, the chatbot uses some prepared answers (by intent)

In [17]:
def get_answer_by_intent(intent):
    if intent in BOT_CONFIG['intents']:
        responses = BOT_CONFIG['intents'][intent]['responses']
        if responses:
          return random.choice(responses)

## Development of NER functionality

Let's load a model which is based on a Spacy model and trained on **MultiWoZ** dataset, especially on the hotel related dialogues.


In [18]:


#load the model
nlp = spacy.load(NER_MODEL_DIR)
# nlp = spacy.load('en_core_web_lg')
# nlp = spacy.load('da_core_news_lg')

ner = nlp.get_pipe('ner')

Let us create functions, which will help to identify date and location.
It can be difficult to identify location only on danish dataset, so I made a translation of danish text into English and used built-in function GeoText.

In [19]:
# Utility functions to get and print output entities
def get_ents(nlp, text):
  docx = nlp(text)
  out = []
  for token in docx.ents:
      out.append((token.text, token.start_char, token.end_char,token.label_))
  return out

def print_ents(nlp, text):
  docx = nlp(text)
  for token in docx.ents:
      print("text:{}, start:{}, end:{}, label:{}".format(token.text,token.start_char, token.end_char,token.label_))

# Translate text
def translate(text, target='en'):
  if type(text) == list:
    out = []
    for t in text:
      # print(t)
      out.append(GoogleTranslator(source='auto', target=target).translate(t))
    return out
  return GoogleTranslator(source='auto', target=target).translate(text)

# Date extracting function
def extract_date(user_message, ents):
  dates = []
  for ent in ents:
    if ent[3] == 'DATE':
      parsed = dateparser.parse(ent[0])
      if parsed:
        dates.append(parsed)
  # try to translate the message to english and use parser
  translated_user_message = translate(user_message, target='en')

  chars = "\|/`*_{}[]()>#+-.,:;!$"
  for c in chars:
        translated_user_message = translated_user_message.replace(c, ' to ')
  # print(translated_user_message)
           # a generator will be returned by the datefinder module. 
          # I'm typecasting it to a list. 
  matches = list(datefinder.find_dates(translated_user_message))
  dates += matches
  # print(str(dates[0].date()))
  return sorted(dates)



#translate text to english and extract cities then return to source language
def extract_loc (src_text, ents=None):
  eng_text = translate(src_text, target='en')
  eng_places = GeoText(eng_text)
  # print(eng_places)
  places = translate(eng_places.cities, target='da')
  
  if ents:
    for ent in ents:
      if ent[3] == 'GPE' or ent[3] == 'LOC':
        places.append(ent[0])
  
  # print(places.cities)
  return  list(set(places))



In [20]:
# test
print('entities: ', get_ents(nlp, "Jeg elsker Paris, men jeg kan ikke lide Frankfurt. 5. januar skal jeg besøge. december 8skal jeg besøge"))
s = 'Jeg elsker Paris, men jeg kan ikke lide Frankfurt. 5. januar skal jeg besøge.'
ents = get_ents(nlp, s)
print(extract_date(s, ents ), ents)
print(extract_loc(s, ents ), ents)

# print_ents(nlp, "Har du mulighed at finde et København?")

2021-05-30 18:06:07,028 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): translate.google.com:443
2021-05-30 18:06:07,077 - urllib3.connectionpool - DEBUG - https://translate.google.com:443 "GET /m?hl=en&sl=auto&q=Jeg+elsker+Paris%2C+men+jeg+kan+ikke+lide+Frankfurt.+5.+januar+skal+jeg+bes%C3%B8ge. HTTP/1.1" 200 None
2021-05-30 18:06:07,151 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): translate.google.com:443
2021-05-30 18:06:07,202 - urllib3.connectionpool - DEBUG - https://translate.google.com:443 "GET /m?hl=en&sl=auto&q=Jeg+elsker+Paris%2C+men+jeg+kan+ikke+lide+Frankfurt.+5.+januar+skal+jeg+bes%C3%B8ge. HTTP/1.1" 200 None


entities:  []
[datetime.datetime(2021, 1, 5, 0, 0)] []


2021-05-30 18:06:07,289 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): translate.google.com:443
2021-05-30 18:06:07,320 - urllib3.connectionpool - DEBUG - https://translate.google.com:443 "GET /m?hl=da&sl=auto&q=Paris HTTP/1.1" 200 None
2021-05-30 18:06:07,398 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): translate.google.com:443
2021-05-30 18:06:07,427 - urllib3.connectionpool - DEBUG - https://translate.google.com:443 "GET /m?sl=auto&q=Paris&tl=da HTTP/1.1" 200 None


['Paris'] []


## Development of a generative model

In [45]:
class CharRNN(nn.Module):
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        self.vocab_size = len(tokens)
        self.input_dim = n_hidden
        self.output_size = len(tokens)
        self.embedding = nn.Embedding(self.vocab_size, self.vocab_size)

        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        # Define the LSTM layer
        self.lstm = lstm_layer = nn.LSTM(self.input_dim, n_hidden, n_layers, dropout=drop_prob, batch_first=True)
        # Define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        self.rnn = getattr(nn, 'LSTM')(len(self.chars), n_hidden, n_layers, dropout=drop_prob, batch_first=True)

        # Define the final, fully-connected output layer        
        self.fc = nn.Linear(self.n_hidden, self.output_size)
        self.sigmoid = nn.Sigmoid()

        self.decoder = nn.Linear(n_hidden, len(self.chars))

        self.init_weights()
        self.cuda()
        
    def forward(self, x, hidden):
        ''' Forward pass through the network '''
        x, h = self.rnn(x, hidden)
        x = self.dropout(x)
        # x = x.view(x.size(0)*x.size(1), self.n_hidden)
        x = x.reshape(x.size(0)*x.size(1), self.n_hidden)
        x = self.decoder(x)

        return x, h

    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

    def init_weights(self):
        ''' Initialize weights of decoder (fully connected layer) '''

        # Apply bias tensor to all zeros
        self.decoder.bias.data.fill_(0)

        # Apply random uniform weights to decoder
        self.decoder.weight.data.uniform_(-1, 1)

        

In [46]:
def one_hot_encode(arr, n_labels):
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [47]:
import json
def load_json(directory, file):
  with open(f'{directory}/{file}') as f:
    db = json.load(f)
  return db

def upload_json(directory, file, db):
  with open(f'{directory}/{file}', mode='w') as f:
    json.dump(db,f)

In [48]:
chars = load_json('/content/drive/MyDrive/ColabNotebooks/MultiWoz', 'chars.txt')


In [49]:
# Set model hyperparameters

n_hidden = 256
n_layers = 5

# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

net = CharRNN(chars, n_hidden, n_layers)
print(net)
net.load_state_dict(torch.load(GENERATIVE_MODEL_SH))
net.eval()

Training on GPU
CharRNN(
  (embedding): Embedding(97, 97)
  (lstm): LSTM(256, 256, num_layers=5, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (rnn): LSTM(97, 256, num_layers=5, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=256, out_features=97, bias=True)
  (sigmoid): Sigmoid()
  (decoder): Linear(in_features=256, out_features=97, bias=True)
)


CharRNN(
  (embedding): Embedding(97, 97)
  (lstm): LSTM(256, 256, num_layers=5, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (rnn): LSTM(97, 256, num_layers=5, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=256, out_features=97, bias=True)
  (sigmoid): Sigmoid()
  (decoder): Linear(in_features=256, out_features=97, bias=True)
)

In [50]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [86]:
def sample(net, size = 500, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)
    answer = ''.join(chars)
    # print(answer)
    a = re.search(r'\b(Person 2)\b', answer)
    slice1 = a.start() + 9
    answer = answer[slice1:]
    # print(answer)
    try:
      b = re.search(r'\b(Person 1)\b', answer)
      slice2 = b.start() 

      # print(answer, answer[slice1 : slice2])
      return answer[: slice2]
    except:
      return answer

def generate_answer(replica):
  return sample(net, size = 500, prime = replica, top_k=3)

In [96]:
print(sample(net, 300, prime= 'Jeg skal finde er restaurant til at spise', top_k=5))

 Okay, jeg har tog TR7199 og ankommer kl. 15:21, og køkken, hvilken dag rejser du? 


# ChatBot response function

In [98]:

context_dict ={} # {chat_id:{'text':[], 'dates':[], 'locs':[], 'last_session':datetime() }}

def clear_context_dict(chat_id):
  return {'text':[], 'dates':[], 'locs':[], 'ents':[], 'last_session': datetime.today(), 'failure_qty':0 }

def bot_responses(input_text, update, chat_id):
  # store all messages in context_dict
  context_dict.setdefault(chat_id, clear_context_dict(chat_id))
  context_dict[chat_id]['text'].append(update.message.text)
  
  # NER and add new entitie to entity_dict
  ents = get_ents(nlp, input_text)

  dates = extract_date(input_text, ents)
  locations = extract_loc (input_text, ents)
  
  context_dict[chat_id]['ents'] += ents
  context_dict[chat_id]['dates'] += dates
  context_dict[chat_id]['locs'] += locations

  #Check last session
  if datetime.today() - context_dict[chat_id]['last_session'] > timedelta( hours=20):
    context_dict[chat_id] = clear_context_dict(chat_id)

  
  # remove copies
  context_dict[chat_id]['locs'] = list(set(context_dict[chat_id]['locs']))
  context_dict[chat_id]['dates'] = list(set(context_dict[chat_id]['dates']))
  print('context_dict:', context_dict)
  entities_qty = len(context_dict[chat_id]['locs']) + len(context_dict[chat_id]['dates'])


  if entities_qty > 3:
    context_dict[chat_id] = clear_context_dict(chat_id)
    # context_dict[chat_id]['failure_qty'] += 1
    return too_many_enity_failure_phrase()

# booking API
  elif entities_qty == 3:
    if len(context_dict[chat_id]['locs'])==1 and len(context_dict[chat_id]['dates'])==2:
      city = context_dict[chat_id]['locs'][-1]
      dates_travel = context_dict[chat_id]['dates']
      context_dict[chat_id] = clear_context_dict(chat_id)
      return booking_request(city, dates_travel)
    else:
      context_dict[chat_id] = clear_context_dict(chat_id)
      context_dict[chat_id]['failure_qty'] += 1
      return too_many_enity_failure_phrase()
# generate answer
  elif context_dict[chat_id]['failure_qty'] >= 3:
  # Using generative model
    # answer = generate_answer(user_message)
    # if answer:
    #     return answer + user_message + out_text
    return generate_answer(input_text)
  else:
    # NLU
    user_message = clear(input_text)
    intent = classify_intent(user_message)
    # print(intent)

    # Answer generation - choose an answer
    if intent:
        answer = get_answer_by_intent(intent)
        if answer:
            return answer



  # failure answer if bot doesn't catch a user
    context_dict[chat_id]['failure_qty'] += 1
    return get_failure_phrase()







## ChatBot commands

In [63]:
def start_command(update, context):
  update.message.reply_text('Jeg er AirBnb Bot, jeg kan hjælpe dig med at finde et hotel i alle lande. Skriv noget til at begynde!')

def help_command(update, context):
  update.message.reply_text('Hvis du har brugt for hjælp, kan du ringe til +1222-345-6789')


def handle_message(update, context):
  chat_id = update.message.chat_id
  print(update, context)
  print(update.message, context)
  text = str(update.message.text)
  response = bot_responses(text, update, chat_id)
  print(response)
  if type(response) == list:
    update.message.reply_text('Her kan du se flere hoteller, der måske passer dig! Hav en rigtig god dag!')
    for elem in response:
      text=elem['hotel_name'] + ' (Bedømmelse: ' + str(elem['review_score']) + ') minimumspris=' + str(elem['min_total_price']) + ' ' + elem['currencycode']
      update.message.reply_text(text,
                                reply_markup = telegram.InlineKeyboardMarkup([
                                    [telegram.InlineKeyboardButton(text='link', url=elem['url'])],
                                ]))
      # update.message.send_photo(chat_id=chat_id,
      #                           photo=elem['main_photo_url']
      #                       )


  else:
    update.message.reply_text(response)

def error(update, context):
  print(f'Update {update} caused error {context.error}')

def main():
  updater = Updater(BOT_KEY, use_context = True)
  dp = updater.dispatcher
  dp.add_handler(CommandHandler('start', start_command))
  dp.add_handler(CommandHandler('help', help_command))
  dp.add_handler(MessageHandler(Filters.text, handle_message))
  dp.add_error_handler(error)
  updater.start_polling()
  logger = logging.getLogger()
  logger.setLevel(logging.INFO)
  updater.idle()



# Call the ChatBot

In [97]:
main()

2021-05-30 18:54:56,141 - apscheduler.scheduler - INFO - Scheduler started


{'update_id': 739756797, 'message': {'message_id': 733, 'date': 1622400921, 'chat': {'id': 756484216, 'type': 'private', 'username': 'lsharaborina', 'first_name': 'lsharaborina'}, 'text': 'Hvilke bog kan du lige?', 'entities': [], 'caption_entities': [], 'photo': [], 'new_chat_members': [], 'new_chat_photo': [], 'delete_chat_photo': False, 'group_chat_created': False, 'supergroup_chat_created': False, 'channel_chat_created': False, 'from': {'id': 756484216, 'first_name': 'lsharaborina', 'is_bot': False, 'username': 'lsharaborina', 'language_code': 'ru'}}} <telegram.ext.callbackcontext.CallbackContext object at 0x7f9e201f8b10>
{'message_id': 733, 'date': 1622400921, 'chat': {'id': 756484216, 'type': 'private', 'username': 'lsharaborina', 'first_name': 'lsharaborina'}, 'text': 'Hvilke bog kan du lige?', 'entities': [], 'caption_entities': [], 'photo': [], 'new_chat_members': [], 'new_chat_photo': [], 'delete_chat_photo': False, 'group_chat_created': False, 'supergroup_chat_created': Fals

2021-05-30 18:55:56,968 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2021-05-30 18:55:56,973 - apscheduler.scheduler - INFO - Scheduler has been shut down
